In [152]:
import oandapyV20.endpoints.instruments as instruments
import oandapyV20.endpoints.pricing as pricing
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.trades as trades
import pandas as pd
import oandapyV20 

In [153]:
# Account information
OANDA_ACCESS_TOKEN = "987464a00c47a9b186fcc7a93a9404a6-bf2ecb97ae681e4edeb529adef404b09"
ACCOUNT_ID = "101-001-8028197-001"
access_token = OANDA_ACCESS_TOKEN
accountID = ACCOUNT_ID
client = oandapyV20.API(access_token=access_token)
API_KEY = access_token

In [154]:
# Instrument (USDJPY)
instrument = "USD_JPY"

# Granularity (daily)
granularity = "D"

# Moving average periods
short_ma_period = 2
long_ma_period = 10

# Risk management
stop_loss_pips = 30
take_profit_pips = 60


In [155]:

# OANDA API connection setup
api = API(access_token=API_KEY, environment="practice")

# Function to fetch historical data
def get_historical_data(instrument, granularity, start, end):
    params = {
        "granularity": granularity,
        "from": start,
        "to": end,
    }
    request = instruments.InstrumentsCandles(instrument=instrument, params=params)
    response = api.request(request)
    data = response['candles']
    ohlc_data = [{'time': candle['time'], 'open': float(candle['mid']['o']), 'high': float(candle['mid']['h']),
                  'low': float(candle['mid']['l']), 'close': float(candle['mid']['c'])} for candle in data]
    df = pd.DataFrame(ohlc_data)
    df['time'] = pd.to_datetime(df['time'])
    df.set_index('time', inplace=True)
    return df

# Define the instrument, granularity, and date range
instrument = 'USD_JPY'
granularity = 'D'  # Daily data
start_date = '2022-01-01T00:00:00Z'
end_date = '2023-01-01T00:00:00Z'

# Fetch historical data
historical_data = get_historical_data(instrument, granularity, start_date, end_date)
data = historical_data
# Print the fetched data
#print(historical_data)
#display (data)

In [156]:
# Add on to Data Dataframe for Profit, Units, Balance, 2 day moving average, 10 day moving average 

# Pivot calculation 
pivot= ((data['open']+data ['low'] + data['close'])/3)
data['pivot'] = pivot          
data['previous_pivot'] = data['pivot'].shift(1)

# Create a Signal column with 1, whenever the previous pivot is less than current open 
data.loc[data['previous_pivot'] < data['open'], 'Signal'] = 1

display(data)


,open,high,low,close,pivot,previous_pivot,Signal
time,,,,,,,
2022-01-02 22:00:00+00:00,115.118,115.370,114.950,115.350,115.139333,NaN,NaN
2022-01-03 22:00:00+00:00,115.328,116.352,115.292,116.148,115.589333,115.139333,1.0
2022-01-04 22:00:00+00:00,116.102,116.244,115.621,116.105,115.942667,115.589333,1.0
2022-01-05 22:00:00+00:00,116.119,116.186,115.625,115.844,115.862667,115.942667,1.0
2022-01-06 22:00:00+00:00,115.852,116.048,115.540,115.549,115.647000,115.862667,NaN
...,...,...,...,...,...,...,...
2022-12-22 22:00:00+00:00,132.416,133.145,132.154,132.827,132.465667,132.147000,1.0
2022-12-26 22:00:00+00:00,132.857,133.598,132.634,133.506,132.999000,132.465667,1.0
2022-12-27 22:00:00+00:00,133.490,134.500,133.396,134.486,133.790667,132.999000,1.0


In [157]:
# Only show days with signal generated 
data[data['Signal'] == 1]
df = data    
# drop any NAN or NA values, filter the list of trades 
df = df.dropna()      
display(df)

,open,high,low,close,pivot,previous_pivot,Signal
time,,,,,,,
2022-01-03 22:00:00+00:00,115.328,116.352,115.292,116.148,115.589333,115.139333,1.0
2022-01-04 22:00:00+00:00,116.102,116.244,115.621,116.105,115.942667,115.589333,1.0
2022-01-05 22:00:00+00:00,116.119,116.186,115.625,115.844,115.862667,115.942667,1.0
2022-01-11 22:00:00+00:00,115.298,115.487,114.382,114.638,114.772667,115.208333,1.0
2022-01-16 22:00:00+00:00,114.168,114.653,114.164,114.612,114.314667,113.951000,1.0
...,...,...,...,...,...,...,...
2022-12-21 22:00:00+00:00,132.423,132.724,131.644,132.374,132.147000,131.868333,1.0
2022-12-22 22:00:00+00:00,132.416,133.145,132.154,132.827,132.465667,132.147000,1.0
2022-12-26 22:00:00+00:00,132.857,133.598,132.634,133.506,132.999000,132.465667,1.0


In [158]:
# Here we create the buy order 
# If signal 1, Buy X amount of Units at the close, Declare Stop loss and profit target, calculate returns, balance and profit 